In [38]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from IPython.display import display
import re
#from nltk.corpus import stopwords
from sklearn.metrics import log_loss
import string

#stopwords = stopwords.words('english')


In [39]:
data = {'query':['Which Team became the 6th place in 2006?','Which Team was world champion in the year 2010?', 'Who won in 1990', 'Who became the world champion in 1945',
        'Which place did germany become in 1986', 'Which place became germany in 2014?','Which place was morroco in 2022?',
        'Who hosted the world cup in the year 2018?','Which nation was the host of the world cup in 1950?','Where was the world cup 1958',
        'in average, how many goals wher scored in 1998?','How many goals where scored 1962 on average', 'averagd Goals scored 1986',
        'How many goals where scored 1950?','How much goals in 2010?','Number of overall goals in 2022',
        'How many matches where played in 1930?', 'Number of matches played in 1954','Amount of Matches in the 2010 Worldcup'],


       'category':['PlacementTeam','PlacementTeam','PlacementTeam','PlacementTeam',
       'TeamPlacement','TeamPlacement','TeamPlacement',
       'YearHost','YearHost','YearHost',
       'year(avg)Goals','year(avg)Goals', 'year(avg)Goals',
       'yearGoals','yearGoals','yearGoals',
       'yearMatches','yearMatches','yearMatches']}

In [40]:
df = pd.DataFrame(data)
df

,query,category
0,Which Team became the 6th place in 2006?,PlacementTeam
1,Which Team was world champion in the year 2010?,PlacementTeam
2,Who won in 1990,PlacementTeam
3,Who became the world champion in 1945,PlacementTeam
4,Which place did germany become in 1986,TeamPlacement
5,Which place became germany in 2014?,TeamPlacement
6,Which place was morroco in 2022?,TeamPlacement
7,Who hosted the world cup in the year 2018?,YearHost
8,Which nation was the host of the world cup in ...,YearHost
9,Where was the world cup 1958,YearHost


In [41]:
print('Query of sample:', df['query'][0])
print('Category of sample:', df['category'][0])

Query of sample: Which Team became the 6th place in 2006?
Category of sample: PlacementTeam


Spacy Cleaning and Preprocessing

In [47]:
import spacy
import nltk
nltk.download('stopwords')
nlp = spacy.load('en_core_web_sm')
punctuations = string.punctuation
#stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pascalbreucker/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [48]:
# Clean text before feeding it to spaCy
stopwords = stopwords
# Define function to cleanup text by removing personal pronouns, stopwords, and puncuation
def cleanup_text(docs, logging=False):
    texts = []
    counter = 1
    for doc in docs:
        if counter % 1000 == 0 and logging:
            print("Processed %d out of %d documents." % (counter, len(docs)))
        counter += 1
        doc = nlp(doc, disable=['parser', 'ner'])
        tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != '-PRON-']
        tokens = [tok for tok in tokens if tok not in stopwords and tok not in punctuations]
        tokens = ' '.join(tokens)
        texts.append(tokens)
    return pd.Series(texts)

In [49]:
# Cleanup text and make sure it retains original shape
print('Original training data shape: ', df['query'].shape)
train_cleaned = cleanup_text(df['query'], logging=True)
print('Cleaned up training data shape: ', train_cleaned.shape)

Original training data shape:  (19,)
Cleaned up training data shape:  (19,)


In [50]:
# Define function to preprocess text for a word2vec model
def cleanup_text_word2vec(docs, logging=False):
    sentences = []
    counter = 1
    for doc in docs:
        if counter % 1000 == 0 and logging:
            print("Processed %d out of %d documents" % (counter, len(docs)))
        # Disable tagger so that lemma_ of personal pronouns (I, me, etc) don't getted marked as "-PRON-"
        doc = nlp(doc, disable=['tagger'])
        # Grab lemmatized form of words and make lowercase
        doc = " ".join([tok.lemma_.lower() for tok in doc])
        # Split into sentences based on punctuation
        doc = re.split("[\.?!;] ", doc)
        # Remove commas, periods, and other punctuation (mostly commas)
        doc = [re.sub("[\.,;:!?]", "", sent) for sent in doc]
        # Split into words
        doc = [sent.split() for sent in doc]
        sentences += doc
        counter += 1
    return sentences


In [51]:
train_cleaned_word2vec = cleanup_text_word2vec(df['query'], logging=True)
print('Cleaned up training data size (i.e. number of sentences): ', len(train_cleaned_word2vec))

Cleaned up training data size (i.e. number of sentences):  19


/Users/pascalbreucker/opt/anaconda3/lib/python3.8/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [52]:
corpus = [" ".join(i) for i in train_cleaned_word2vec]

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer 
vectorizer = TfidfVectorizer()
vec_fit = vectorizer.fit(corpus)
# print(X.toarray())
print(vectorizer.get_feature_names())


text = "login is so cool"
Y = vec_fit.transform([text])
print(Y.toarray())

['1930', '1945', '1950', '1954', '1958', '1962', '1986', '1990', '1998', '2006', '2010', '2014', '2018', '2022', '6th', 'amount', 'averagd', 'average', 'became', 'become', 'champion', 'cup', 'did', 'germany', 'goals', 'host', 'hosted', 'how', 'in', 'many', 'matches', 'morroco', 'much', 'nation', 'number', 'of', 'on', 'overall', 'place', 'played', 'scored', 'team', 'the', 'was', 'wher', 'where', 'which', 'who', 'won', 'world', 'worldcup', 'year']
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]]


Gensim for word embedding

In [54]:
import gensim

In [55]:
# Define function to create word vectors given a cleaned piece of text.
def create_average_vec(doc):
    average = np.zeros(len(vectorizer.get_feature_names()), dtype='float32')
    num_words = 0.
    for word in doc.split():
        average = np.add(average, vec_fit.transform([word]).toarray())
        num_words += 1.
    if num_words != 0.:
        average = np.divide(average, num_words)
    return average

In [56]:
print(create_average_vec("what team won 1990, Germany "))

[[0.  0.  0.  0.  0.  0.  0.  0.2 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.2 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.2 0.  0.  0.  0.  0.  0.  0.2 0.  0.  0. ]]


In [57]:
# Create word vectors
train_cleaned_vec = np.zeros((df.shape[0], len(vectorizer.get_feature_names())), dtype="float32")  # 19579 x 300
for i in range(len(train_cleaned)):
    train_cleaned_vec[i] = create_average_vec(train_cleaned[i])

print("Train word vector shape:", train_cleaned_vec.shape)

Train word vector shape: (19, 52)


In [58]:
# printing the first element of the train_cleaned vector
train_cleaned[0]

'team become 6th place 2006'

Label Encoding

In [59]:
from sklearn.preprocessing import LabelEncoder
Encoder = LabelEncoder()
y_train = Encoder.fit_transform(df['category'])

# looking the label encoding representation of the category column in the dataset
y_train

array([0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5])

In [60]:
Encoder.inverse_transform([0,1,2,3,4])

array(['PlacementTeam', 'TeamPlacement', 'YearHost', 'year(avg)Goals',
       'yearGoals'], dtype=object)

SVM and Grid search

In [66]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
  
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 
svclassifier = SVC(probability = True)
svclassifier.fit(train_cleaned_vec, y_train)
grid = GridSearchCV(svclassifier, param_grid, refit = True, verbose = 3)
grid.fit(train_cleaned_vec, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


ValueError: n_splits=5 cannot be greater than the number of members in each class.